In [1]:
%pip install datasets
%pip install nltk
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Part 0. Data Prepraration

In [2]:
from datasets import load_dataset
dataset = load_dataset("rotten_tomatoes")
train_dataset = dataset ['train']
validation_dataset = dataset ['validation']
test_dataset = dataset ['test']

c:\Users\JJWX\anaconda3\envs\NLP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
len(train_dataset['text'])

8530

# Part 1. Preparing Word Embeddings

#### (a) What is the size of the vocabulary formed from your training data?

In [41]:
import nltk
import numpy as np
nltk.download('punkt')
vocab = set()
for text in train_dataset['text']:
    ls = nltk.word_tokenize(text)
    for word in ls:
        vocab.add(word)
#Size of the vocabulary: 15812

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JJWX\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [42]:
print("Size of the vocabulary:", len(vocab))

Size of the vocabulary: 18030


#### (b) We use OOV (out-of-vocabulary) to refer to those words appeared in the training data but not in the Word2vec (or Glove) dictionary. How many OOV words exist in your training data?

In [43]:
import gensim.downloader as api
for key in api.info()['models'].keys():
    print(key)

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [63]:
model = api.load("glove-wiki-gigaword-100")

In [64]:
oov_words = set()
for word in vocab:
    if word not in model:
        oov_words.add(word)

print("Number of OOV words:", len(oov_words))

Number of OOV words: 1867


#### (c) The existence of the OOV words is one of the well-known limitations of Word2vec (or Glove). Without using any transformer-based language models (e.g., BERT, GPT, T5), what do you think is the best strategy to mitigate such limitation? Implement your solution in your source code. Show the corresponding code snippet.

In [65]:
# Group any words that are not in the model into a single token
def wordtovec(word):
    if word in model:
        return model[word]
    else:
        return np.zeros(model.vector_size)

# Part 2. Model Training & Evaluation - RNN

In [66]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import random

In [ ]:
vocab_size = len(model.index_to_key) + 1
embedding_dim = model.vector_size
word_index = {word: index+1 for index, word in enumerate(model.index_to_key)} # index 0 is reserved for padding
embedding_matrix = np.zeros((vocab_size, embedding_dim))

In [68]:
for word, idx in word_index.items():
    if word in model:
        embedding_matrix[idx] = model[word]

In [ ]:
def tokenize(text, word_index):
    ls = nltk.word_tokenize(text)
    return [word_index[word] for word in ls if word in word_index]

X_train = [tokenize(text, word_index) for text in train_dataset['text']]
X_val = [tokenize(text, word_index) for text in validation_dataset['text']]
X_test = [tokenize(text, word_index) for text in test_dataset['text']]
max_length = max(len(seq) for seq in X_train)

In [104]:
X_train = pad_sequences(X_train, maxlen=max_length, padding='pre', truncating='pre')
X_val = pad_sequences(X_val, maxlen=max_length, padding='pre', truncating='pre')
X_test = pad_sequences(X_test, maxlen=max_length, padding='pre', truncating='pre')

In [ ]:
y_train = np.array(train_dataset['label'])
y_val = np.array(validation_dataset['label'])
y_test = np.array(test_dataset['label'])

In [82]:
X_train

array([[[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  1, ..., -1,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       [[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       [[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       ...,

       [[ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        ...,
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0

Model Training - Grid Search

In [79]:
from tensorflow.keras.callbacks import Callback
best_accuracy = {}
class CustomCallback(Callback):
    accuracy = 0
    cur_key = ""
    epochs = 0
    optimizer = ""
    batch_size = 0
    best_model = None
    lr = 0
    def on_train_begin(self, logs=None):
        self.accuracy = 0

    def on_train_end(self, logs=None):
        global best_accuracy
        if self.accuracy > best_accuracy.get("accuracy", 0):
            best_accuracy = {
                "accuracy": self.accuracy,
                "epoch": self.epochs,
                "optimizer": self.optimizer,
                "batch_size": self.batch_size,
                "lr": self.lr
            }
            print("Saved best accuracy for current run:", self.accuracy, "at epoch", self.epochs)
            self.best_model.save(filepath="best_model.keras")
        print("Run completed on:")
        print(self.cur_key)
        print("Best accuracy for current run:", self.accuracy, "at epoch", self.epochs)
        print("Training ended")


    
    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs['val_accuracy']
        if val_accuracy > self.accuracy:
            self.accuracy = val_accuracy
            self.epochs = epoch
            self.best_model = self.model

    def set_key(self, optimizer, batch_size, lr):
        self.optimizer = optimizer
        self.batch_size = batch_size
        self.lr = lr
        self.cur_key = f"optimizer: {optimizer}, batch_size: {batch_size}, lr: {lr}"

In [80]:
from tensorflow.keras.layers import Dropout
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    # early_stopping = tf.keras.callbacks.EarlyStopping(
    #     monitor='val_accuracy',
    #     patience=3,
    #     restore_best_weights=True
    # )
    custom_callback = CustomCallback()
    custom_callback.set_key(optimizer, batch_size, lr)
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  
        SimpleRNN(32, return_sequences=False),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[custom_callback],
        shuffle=True
    )
    return model, history

In [81]:
for batch_size in [16, 32, 64, 128]:
    for lr in [0.005, 0.01, 0.05, 0.1]:
        for optimizer in ['adam', 'sgd', 'rmsprop', 'adagrad']:
            train_model(optimizer, 100, batch_size, lr)

Epoch 1/100


ValueError: Input 0 of layer "simple_rnn" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 62, 100, 100)

In [35]:
best_accuracy

{'accuracy': 0.7016885280609131,
 'epoch': 35,
 'optimizer': 'adagrad',
 'batch_size': 64,
 'lr': 0.01}

Best model is trained with Optimizer: adagrad, Epoch: 35, Batch_size: 64, Learning_rate: 0.01

In [91]:
model, history = train_model("adagrad", 40, 64, 0.01)

Epoch 1/40
  2/134 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4062 - loss: 0.7197    

KeyboardInterrupt: 

In [93]:
best_model = tf.keras.models.load_model("best_model.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8146 - loss: 0.5822
Test accuracy: 0.501876175403595


#### Mean pooling

In [94]:
from tensorflow.keras.layers import AveragePooling1D, GlobalAveragePooling1D
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_mean.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=True),
        GlobalAveragePooling1D(),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [95]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
133/134 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4691 - loss: 0.6987
Epoch 1: val_accuracy improved from -inf to 0.47842, saving model to model_mean.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.4691 - loss: 0.6987 - val_accuracy: 0.4784 - val_loss: 0.6947
Epoch 2/100
133/134 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4769 - loss: 0.6949
Epoch 2: val_accuracy improved from 0.47842 to 0.49719, saving model to model_mean.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.4770 - loss: 0.6949 - val_accuracy: 0.4972 - val_loss: 0.6932
Epoch 3/100
130/134 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4916 - loss: 0.6935
Epoch 3: val_accuracy improved from 0.49719 to 0.51501, saving model to model_mean.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.4920 - loss: 0.6935 - val_accuracy: 0.5150 - val_loss: 0.6921
Epoch 4/100
131/134 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5066 - loss: 0.6924
Epoch 4: val_accuracy improved fro

In [96]:
best_model = tf.keras.models.load_model("model_mean.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6120 - loss: 0.6263
Test accuracy: 0.6454033851623535


#### Max pooling

In [61]:
from tensorflow.keras.layers import MaxPooling1D, GlobalMaxPooling1D
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_max.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=True),
        GlobalMaxPooling1D(),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [62]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
127/134 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5012 - loss: 0.7078
Epoch 1: val_accuracy improved from -inf to 0.53189, saving model to model_max.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5016 - loss: 0.7073 - val_accuracy: 0.5319 - val_loss: 0.6873
Epoch 2/100
128/134 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5218 - loss: 0.6920
Epoch 2: val_accuracy improved from 0.53189 to 0.55347, saving model to model_max.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.5222 - loss: 0.6919 - val_accuracy: 0.5535 - val_loss: 0.6841
Epoch 3/100
129/134 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5456 - loss: 0.6887
Epoch 3: val_accuracy improved from 0.55347 to 0.56754, saving model to model_max.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.5457 - loss: 0.6887 - val_accuracy: 0.5675 - val_loss: 0.6809
Epoch 4/100
133/134 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5539 - loss: 0.6849
Epoch 4: val_accuracy improved from 0

In [64]:
best_model = tf.keras.models.load_model("model_max.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7259 - loss: 0.5401
Test accuracy: 0.7354596853256226


#### Dense layer

In [58]:
from tensorflow.keras.layers import Flatten
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_dense.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=True),
        Flatten(),
        Dense(62, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [59]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.5358 - loss: 0.6983
Epoch 1: val_accuracy improved from -inf to 0.58349, saving model to model_dense.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 26s 180ms/step - accuracy: 0.5360 - loss: 0.6983 - val_accuracy: 0.5835 - val_loss: 0.6724
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step - accuracy: 0.6117 - loss: 0.6588
Epoch 2: val_accuracy improved from 0.58349 to 0.65009, saving model to model_dense.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 28s 208ms/step - accuracy: 0.6118 - loss: 0.6587 - val_accuracy: 0.6501 - val_loss: 0.6379
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.6471 - loss: 0.6261
Epoch 3: val_accuracy improved from 0.65009 to 0.66886, saving model to model_dense.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 20s 149ms/step - accuracy: 0.6472 - loss: 0.6261 - val_accuracy: 0.6689 - val_loss: 0.6154
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.6783 - loss: 0.6012
Epoch 4: val_accu

In [60]:
best_model = tf.keras.models.load_model("model_dense.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7367 - loss: 0.5282
Test accuracy: 0.7373358607292175


# Part 3. Enhancement


#### 1. Instead of keeping the word embeddings fixed, now update the word embeddings (the same way as model parameters) during the training process.


In [ ]:
tf.random.set_seed(0)
np.random.seed(0)
random.seed(0)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True
)
model = Sequential([
    Embedding(input_dim=vocab_size,
              output_dim=embedding_dim,
              weights=[embedding_matrix],
              trainable=True),
    SimpleRNN(16, return_sequences=False),
    Dense(1, activation='sigmoid')
])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=64,
    callbacks=[early_stopping]
)

#### 2. As discussed in Question 1(c), apply your solution in mitigating the influence of OOV words and train your model again.

In [111]:
model = api.load("glove-wiki-gigaword-100")
vocab_size = len(model.index_to_key) + 2 # 0 is reserved for padding, 1 is reserved for OOV
embedding_dim = model.vector_size
word_index = {word: index+2 for index, word in enumerate(model.index_to_key)} # index 0 is reserved for padding
embedding_matrix = np.zeros((vocab_size, embedding_dim))

In [112]:
for word, idx in word_index.items():
    if word in model:
        embedding_matrix[idx] = model[word]

In [113]:
def tokenize(text, word_index):
    ls = nltk.word_tokenize(text)
    return [word_index[word] if word in word_index else 1 for word in ls]

X_train = [tokenize(text, word_index) for text in train_dataset['text']]
X_val = [tokenize(text, word_index) for text in validation_dataset['text']]
X_test = [tokenize(text, word_index) for text in test_dataset['text']]
max_length = max(len(seq) for seq in X_train)

In [114]:
X_train = pad_sequences(X_train, maxlen=max_length)
X_val = pad_sequences(X_val, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)

In [115]:
y_train = np.array(train_dataset['label'])
y_val = np.array(validation_dataset['label'])
y_test = np.array(test_dataset['label'])

In [ ]:
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_oov.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=False),  # Embedding layer is frozen
        SimpleRNN(16, return_sequences=False),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [30]:
train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5133 - loss: 0.6945
Epoch 1: val_accuracy improved from -inf to 0.52439, saving model to model_oov.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5132 - loss: 0.6945 - val_accuracy: 0.5244 - val_loss: 0.6933
Epoch 2/100
126/134 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5323 - loss: 0.6905
Epoch 2: val_accuracy did not improve from 0.52439
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5322 - loss: 0.6906 - val_accuracy: 0.5206 - val_loss: 0.6940
Epoch 3/100
132/134 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5591 - loss: 0.6857
Epoch 3: val_accuracy improved from 0.52439 to 0.53377, saving model to model_oov.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5589 - loss: 0.6858 - val_accuracy: 0.5338 - val_loss: 0.6915
Epoch 4/100
131/134 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5766 - loss: 0.6794
Epoch 4: val_accuracy improved from 0.53377 to 0.55253, saving model to mode

(<Sequential name=sequential_1, built=True>,
 <keras.src.callbacks.history.History at 0x1d5b9a1ec90>)

In [31]:
best_model = tf.keras.models.load_model("model_oov.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6786 - loss: 0.5732
Test accuracy: 0.6763602495193481


#### 3. Keeping the above two adjustments, replace your simple RNN model in Part 2 with a biLSTM model and a biGRU model, incorporating recurrent computations in both directions and stacking multiple layers if possible

BiLSTM

In [67]:
from tensorflow.keras.layers import Bidirectional, LSTM
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_combined.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True),  # Embedding layer is frozen
        Bidirectional(LSTM(16, return_sequences=True)),
        Bidirectional(LSTM(16, return_sequences=True)),
        Bidirectional(LSTM(16, return_sequences=False)),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback]
    )
    return model, history

In [68]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.5006 - loss: 0.6935
Epoch 1: val_accuracy improved from -inf to 0.58443, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 33s 202ms/step - accuracy: 0.5006 - loss: 0.6935 - val_accuracy: 0.5844 - val_loss: 0.6909
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 0.5479 - loss: 0.6905
Epoch 2: val_accuracy improved from 0.58443 to 0.60038, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 24s 180ms/step - accuracy: 0.5479 - loss: 0.6905 - val_accuracy: 0.6004 - val_loss: 0.6876
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.5880 - loss: 0.6862
Epoch 3: val_accuracy improved from 0.60038 to 0.61632, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 25s 188ms/step - accuracy: 0.5880 - loss: 0.6862 - val_accuracy: 0.6163 - val_loss: 0.6805
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.6149 - loss: 0.6756
Epoch 4:

In [65]:
model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_16 (Embedding)        │ (None, 62, 100)        │    40,000,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_11                │ (None, 62, 32)         │        14,976 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_12                │ (None, 62, 32)         │         6,272 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_13                │ (None, 32)             │         6,272 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,055,508 (305.39 MB)

 Trainable params: 40,027,753 (152.69 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 40,027,755 (152.69 MB)

In [66]:
best_model = tf.keras.models.load_model("model_combined.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.6798 - loss: 0.5810
Test accuracy: 0.7429643273353577


BiGRU

In [37]:
from tensorflow.keras.layers import Bidirectional, GRU
def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_combined.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True),  # Embedding layer is frozen
        Bidirectional(GRU(16, return_sequences=False)),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [38]:
train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - accuracy: 0.5007 - loss: 0.7018
Epoch 1: val_accuracy improved from -inf to 0.50750, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 28s 176ms/step - accuracy: 0.5007 - loss: 0.7017 - val_accuracy: 0.5075 - val_loss: 0.6927
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.5439 - loss: 0.6876
Epoch 2: val_accuracy improved from 0.50750 to 0.55159, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 24s 182ms/step - accuracy: 0.5440 - loss: 0.6876 - val_accuracy: 0.5516 - val_loss: 0.6857
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.5801 - loss: 0.6800
Epoch 3: val_accuracy improved from 0.55159 to 0.55910, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 24s 179ms/step - accuracy: 0.5801 - loss: 0.6800 - val_accuracy: 0.5591 - val_loss: 0.6803
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 0.5972 - loss: 0.6738
Epoch 4:

(<Sequential name=sequential_4, built=True>,
 <keras.src.callbacks.history.History at 0x1d5d1d63690>)

In [39]:
best_model = tf.keras.models.load_model("model_combined.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7120 - loss: 0.5646
Test accuracy: 0.7532833218574524


#### 4. Keeping the above two adjustments, replace your simple RNN model in Part 2 with a Convolutional Neural Network (CNN) to produce sentence representations and perform sentiment classification

In [50]:
from tensorflow.keras.layers import Convolution1D, Flatten

def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        restore_best_weights=True
    )
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_combined.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )
    model = Sequential([
        Embedding(input_dim=vocab_size,
                  output_dim=embedding_dim,
                  weights=[embedding_matrix],
                  trainable=True),  # Embedding layer is trainable
        Convolution1D(16, kernel_size=3, activation='relu'),
        Flatten(),
        Dense(1, activation='sigmoid')
    ])
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, early_stopping]
    )
    return model, history

In [52]:
model, history = train_model("adagrad", 100, 64, 0.01)

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.4938 - loss: 0.7037
Epoch 1: val_accuracy improved from -inf to 0.53846, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 19s 133ms/step - accuracy: 0.4939 - loss: 0.7036 - val_accuracy: 0.5385 - val_loss: 0.6903
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.5456 - loss: 0.6871
Epoch 2: val_accuracy improved from 0.53846 to 0.58161, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 18s 131ms/step - accuracy: 0.5457 - loss: 0.6870 - val_accuracy: 0.5816 - val_loss: 0.6789
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.5965 - loss: 0.6690
Epoch 3: val_accuracy improved from 0.58161 to 0.60694, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 18s 131ms/step - accuracy: 0.5966 - loss: 0.6690 - val_accuracy: 0.6069 - val_loss: 0.6650
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 0.6333 - loss: 0.6457
Epoch 4:

In [53]:
best_model = tf.keras.models.load_model("model_combined.keras")
accuracy = best_model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6092 - loss: 0.7034
Test accuracy: 0.7101313471794128


#### 5 

In [106]:
from tensorflow.keras.layers import MultiHeadAttention, Input, Bidirectional, LSTM, Dropout, Dense, Lambda, GlobalMaxPooling1D, Flatten, GlobalAveragePooling1D, GRU
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers

def train_model(optimizer, epochs, batch_size, lr):
    tf.random.set_seed(0)
    np.random.seed(0)
    random.seed(0)
    
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath="model_combined.keras", 
        monitor='val_accuracy',            
        save_best_only=True,           
        mode='max',                 
        save_weights_only=False,       
        verbose=1
    )

    input_layer = Input(shape=(max_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], trainable=True)(input_layer)
    
    gru_output = GRU(32, return_sequences=True)(embedding_layer)
    gru_output = Dropout(0.5)(gru_output)
    attention_output = MultiHeadAttention(num_heads=1, key_dim=16)(gru_output, gru_output)
    gru_output = Bidirectional(GRU(16, return_sequences=True))(attention_output)
    max_output = GlobalMaxPooling1D()(gru_output)
    gru_output = gru_output[:, -1, :]
    concat_output = tf.keras.layers.Concatenate(axis=1)([max_output, gru_output])
    output_layer = Dense(1, activation='sigmoid')(concat_output) 

    # Create the model
    model = Model(inputs=input_layer, outputs=output_layer)

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',     # Monitor validation loss
        factor=0.75,             # Factor to reduce the learning rate
        patience=3,             # Number of epochs with no improvement to wait
        min_lr=1e-6             # Minimum learning rate limit
    )
    # Set optimizer
    if optimizer == 'adam': optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == 'sgd': optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    elif optimizer == 'rmsprop': optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    else: optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)

    # Compile the model
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=[checkpoint_callback, reduce_lr]
    )

    return model, history

In [107]:
model, history = train_model("adagrad", 200, 64, 0.01)

Epoch 1/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.4986 - loss: 0.6936
Epoch 1: val_accuracy improved from -inf to 0.50844, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 26s 164ms/step - accuracy: 0.4986 - loss: 0.6936 - val_accuracy: 0.5084 - val_loss: 0.6930 - learning_rate: 0.0100
Epoch 2/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.5070 - loss: 0.6930
Epoch 2: val_accuracy improved from 0.50844 to 0.52345, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 22s 167ms/step - accuracy: 0.5070 - loss: 0.6930 - val_accuracy: 0.5235 - val_loss: 0.6927 - learning_rate: 0.0100
Epoch 3/200
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - accuracy: 0.5170 - loss: 0.6927
Epoch 3: val_accuracy improved from 0.52345 to 0.53189, saving model to model_combined.keras
134/134 ━━━━━━━━━━━━━━━━━━━━ 23s 170ms/step - accuracy: 0.5170 - loss: 0.6927 - val_accuracy: 0.5319 - val_loss: 0.6924 - learning_rate: 0.0100
Epoch 4/200
134/134 ━━━━━

In [109]:
model = tf.keras.models.load_model("model_combined.keras")
accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy[1])

34/34 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.7717 - loss: 0.4819
Test accuracy: 0.7823639512062073


In [100]:
model.summary()

Model: "functional_59"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_42      │ (None, 62)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_43        │ (None, 62, 100)   │ 40,000,100 │ input_layer_42[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_41 (GRU)        │ (None, 62, 64)    │     31,872 │ embedding_43[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_42 (GRU)        │ (None, 62, 64)    │     31,872 │ embedding_43[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_43 (GRU)        │ (None, 62, 64)    │     31,872 │ embedding_43[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_29 (LSTM)      │ (None, 62, 64)    │     42,240 │ embedding_43[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_30 (LSTM)      │ (None, 62, 64)    │     42,240 │ embedding_43[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_31 (LSTM)      │ (None, 62, 64)    │     42,240 │ embedding_43[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_77          │ (None, 62, 64)    │          0 │ gru_41[0][0]      │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_78          │ (None, 62, 64)    │          0 │ gru_42[0][0]      │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_79          │ (None, 62, 64)    │          0 │ gru_43[0][0]      │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_81          │ (None, 62, 64)    │          0 │ lstm_29[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_82          │ (None, 62, 64)    │          0 │ lstm_30[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_83          │ (None, 62, 64)    │          0 │ lstm_31[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 62, 64)    │     49,792 │ dropout_77[0][0], │
│ (MultiHeadAttentio… │                   │            │ dropout_78[0][0], │
│                     │                   │            │ dropout_79[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 62, 64)    │     33,216 │ dropout_81[0][0], │
│ (MultiHeadAttentio… │                   │            │ dropout_82[0][0], │
│                     │                   │            │ dropout_83[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_55    │ (None, 62, 32)    │      7,872 │ multi_head_atten… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_56    │ (None, 62, 32)    │     10,368 │ multi_head_atten… │
│ (Bidirectional)     │                   │            │                 

 Total params: 80,647,628 (307.65 MB)

 Trainable params: 40,323,813 (153.82 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 40,323,815 (153.82 MB)

In [44]:
train_dataset['text']

['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .',
 'effective but too-tepid biopic',
 'if you sometimes like to go to the movies to have fun , wasabi is a good place to start .',
 "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .",
 'the film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .',
 'offers that rare combination of entertainment and education .',
 'perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions .',
 "steers turns i

In [48]:
len(train_dataset['label'])

8530

In [49]:
sum(train_dataset['label'])

4265